[View in Colaboratory](https://colab.research.google.com/github/KhanradCoder/StanfordAdmissionPredictor/blob/master/stanfordpredictor.ipynb)

# Importing the Data

In [0]:
import numpy as np
import pandas as pd
import csv
import urllib2

url = 'https://raw.githubusercontent.com/KhanradCoder/StanfordAdmissionPredictor/master/stanford_data.csv'
response = urllib2.urlopen(url)
admit = pd.read_csv(response)

In [0]:
from sklearn.model_selection import train_test_split
x = admit.drop('Admit',axis=1)
y_labels = admit['Admit']
assert not np.any(np.isnan(x)) #makes sure there are no missing values in the data

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x, y_labels, test_size=0.2, random_state=101)

In [0]:
import tensorflow as tf
unweight = tf.feature_column.numeric_column("UW")
weight = tf.feature_column.numeric_column("W")
sat = tf.feature_column.numeric_column("SAT")
rank = tf.feature_column.numeric_column("Rank")
feat_cols = [unweight,weight,sat,rank]

# Train the model
Note you may want to iterate through this cell a couple of times to get the most accurate results.


In [141]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=None,shuffle=True)
model = tf.estimator.DNNClassifier(feature_columns=feat_cols,hidden_units=[5,10], model_dir='stanford_models')
for i in range(20):
  model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f627a331350>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'stanford_models', '_train_distribute': None, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from stanford_models/model.ckpt-31000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tens

# Test the model
To see what your chances are, modify the 'my_stats' variable in this format:


```
my_stats = [unweighted_gpa,weighted_gpa,sat,class_percentile_ranking]
```



In [142]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)
predictions = list(model.predict(input_fn=pred_fn))
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])
from sklearn.metrics import classification_report
print(classification_report(y_test,final_preds))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from stanford_models/model.ckpt-231000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
             precision    recall  f1-score   support

          0       0.94      1.00      0.97       140
          1       0.00      0.00      0.00         9

avg / total       0.88      0.94      0.91       149



In [147]:
my_stats = [4,4.8,1600,0.99]

data = pd.DataFrame({'UW': [my_stats[0]], 'W': [my_stats[1]], 'SAT': [my_stats[2]],
                     'Rank': [my_stats[3]]})

pred_fn = tf.estimator.inputs.pandas_input_fn(x=data,num_epochs=1,shuffle=False)

import numpy as np
pred_gen = list(model.predict(input_fn=pred_fn))

likelyhood = pred_gen[0]['logistic']
likelyhood = round(likelyhood[0]*100,2)
print('\n')
print ("You are "+str(likelyhood)+"% likely to get into Stanford")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from stanford_models/model.ckpt-231000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


You are 24.3% likely to get into Stanford
